# GTFS Data Analysis

In [2]:
# Necessary imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import json
import datetime
import re
import math
import plotly.io as pio


Select a railroad:

In [3]:
railroad = "mnrr"

Gather data

In [4]:
df = pd.read_csv(f'../gtfs_data/{railroad}/stops.txt')
df = df[['stop_name','stop_lat','stop_lon']]

df_ref = pd.read_csv(f'../data/csv/{railroad}.csv')
row_stop_counts = df_ref.drop(columns=["stop_name","Unnamed: 0"]).notna().sum(axis=1)
df['counts'] = row_stop_counts
df = df.sort_values(by='counts',ascending=False).reset_index(drop=True)

Draw stop frequency map (all day)

In [5]:
fig = px.scatter_map(df,
                    lat=df.stop_lat,
                    lon=df.stop_lon,
                    hover_name="stop_name",
                    color="counts", 
                    color_continuous_scale="Viridis",
                    range_color=[0, df["counts"].max()]
                   )
fig.update_geos(fitbounds="locations")
fig.update_layout(map_style="carto-darkmatter-nolabels")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [6]:
with open(f'../data/json/data{railroad}.json', 'r') as file:
    train_list = json.load(file)
    if (len(train_list) <= 0):
        print(f"\tWarning! Empty List.")
        print(f"\tNumber of trains: 0")

In [7]:
train_lines = sorted(list({train["train_line"] for train in train_list}))
graph_for_lines = {}
for train_line in train_lines:
    total_train_in_route = sum(1 for train in train_list if train["train_line"] == train_line)
    graph_for_lines[train_line] = total_train_in_route

graph_for_lines = pd.DataFrame(list(graph_for_lines.items()), columns=["train_line", "value"])

# Create the pie chart
fig = px.pie(graph_for_lines, names="train_line", values="value", title="Rides by Line")
fig.show()

In [8]:
num_stops = []
binned_times = {
    "4-10" : 0,
    "10-16" : 0,
    "16-22" : 0,
    "22-4": 0
}
hourly_times = {}
hourly_times_arriving = {}
for train in train_list:
    df = pd.DataFrame(train['stops']).transpose()
    df_cleaned = df[df['departure_time'] != 'n/a']
    df_cleaned.loc[:, 'stop_index'] = df_cleaned['stop_index'].astype(int)
    df_cleaned = df_cleaned.sort_values(by='stop_index')

    timeA = df_cleaned.iloc[0].departure_time
    timeB = df_cleaned.iloc[len(df_cleaned)-1].departure_time

    timeA = f'1900-01-{1+(int(timeA[:timeA.find(":")])//24)} {int(timeA[:timeA.find(":")])%24}:{timeA[timeA.find(":")+1:]}' if re.match(r"^([2-9][4-9]|[3-9]\d|\d{3,}):.*",timeA) else f'1900-01-01 {timeA}'
    timeB = f'1900-01-{1+(int(timeB[:timeB.find(":")])//24)} {int(timeB[:timeB.find(":")])%24}:{timeB[timeB.find(":")+1:]}' if re.match(r"^([2-9][4-9]|[3-9]\d|\d{3,}):.*",timeB) else f'1900-01-01 {timeB}'
    timeA = datetime.datetime.strptime(timeA, '%Y-%m-%d %H:%M:%S')
    timeB = datetime.datetime.strptime(timeB, '%Y-%m-%d %H:%M:%S')
    
    binned_times[list(binned_times.keys())[math.floor((timeA.hour-4)/6)]] = binned_times[list(binned_times.keys())[math.floor((timeA.hour-4)/6)]] + 1
    
    if f'{timeA.hour}' in hourly_times:
        hourly_times[f'{timeA.hour}'][int(train['direction_id'])] = hourly_times[f'{timeA.hour}'][int(train['direction_id'])] + 1
    else:
        arr = [0, 0]
        arr[int(train['direction_id'])] = 1
        hourly_times[f'{timeA.hour}'] = arr

    if f'{timeB.hour}' in hourly_times_arriving:
        hourly_times_arriving[f'{timeB.hour}'][int(train['direction_id'])] = hourly_times_arriving[f'{timeB.hour}'][int(train['direction_id'])] + 1
    else:
        arr = [0, 0]
        arr[int(train['direction_id'])] = 1
        hourly_times_arriving[f'{timeB.hour}'] = arr

    # print(
    #     df_cleaned.iloc[len(df_cleaned)-1].name , train["direction_id"] , df_cleaned.iloc[0].name,
    #     df_cleaned.iloc[len(df_cleaned)-1].name if train["direction_id"] == "0" else df_cleaned.iloc[0].name
    #     )
    
    
    num_stops.append({
        'train_id' : train['train_number'],
        'number_of_stops': len(df_cleaned),
        'first_stop_time' : df_cleaned.iloc[0].departure_time,
        'last_stop_time' : df_cleaned.iloc[len(df_cleaned)-1].departure_time,
        'travel_time': int((timeB-timeA).total_seconds()),
        'distance' : train['distance'],
        'departure_station': df_cleaned.iloc[0].name,
        'arrival_station': df_cleaned.iloc[len(df_cleaned)-1].name,
        'train_line': f'{ train["train_line"]}',
        'train_label': f'{ train["train_number"]} {train["train_line"]} ({df_cleaned.iloc[0].name} - {df_cleaned.iloc[len(df_cleaned)-1].name})'
    })

In [9]:
min_by_stops = min(num_stops, key=lambda x: x["number_of_stops"])
max_by_stops = max(num_stops, key=lambda x: x["number_of_stops"])
trip_durations = [trip['number_of_stops'] for trip in num_stops]
avg_stops = np.mean(trip_durations)
median_stops = np.median(trip_durations)
print(f'\n\tShortest Trip (stop # wise): {min_by_stops["number_of_stops"]} stops on {min_by_stops["train_id"]}')
print(f'\tLongest Trip (stop # wise): {max_by_stops["number_of_stops"]} stops on {max_by_stops["train_id"]}')
print(f'\tOn average trains make: {avg_stops:.0f} stops')
print(f'\tMedian # of train: {median_stops:.0f} stops')
# maybe some sort of normal graph


	Shortest Trip (stop # wise): 4 stops on 507
	Longest Trip (stop # wise): 30 stops on 699
	On average trains make: 13 stops
	Median # of train: 13 stops


In [10]:
if (railroad != "metrolink" and railroad != "sle"):
    cleaned_data = [item for item in num_stops if item["distance"] != "NA"]
    min_by_distance = min(cleaned_data, key=lambda x: x["distance"])
    max_by_distance = max(cleaned_data, key=lambda x: x["distance"])
    trip_durations = [trip['distance'] for trip in cleaned_data]
    # graphs(trip_durations)
    avg_distance = np.mean(trip_durations)
    median_distance = np.median(trip_durations)
    print(f'\n\tShortest Trip (distance wise): {min_by_distance["distance"]:.1f} miles on {min_by_distance["train_id"]}')
    print(f'\tLongest Trip (distance wise): {max_by_distance["distance"]:.1f} miles on {max_by_distance["train_id"]}')
    print(f'\tOn average trains travels: {avg_distance:.1f} miles')
    print(f'\tMedian travel distance: {median_distance:.1f} miles')
    # maybe some sort of normal graph



	Shortest Trip (distance wise): 7.3 miles on 1716
	Longest Trip (distance wise): 82.0 miles on 916
	On average trains travels: 41.7 miles
	Median travel distance: 33.0 miles


In [11]:

min_by_time = min(num_stops, key=lambda x: x["travel_time"])
max_by_time = max(num_stops, key=lambda x: x["travel_time"])
trip_durations = [trip['travel_time'] for trip in num_stops]
avg_time = np.mean(trip_durations)
median_time = np.median(trip_durations)
print(f'\n\tShortest Trip (time wise): {min_by_time["travel_time"]/60:.1f} minutes on {min_by_time["train_id"]}')
print(f'\tLongest Trip (time wise): {max_by_time["travel_time"]/60:.1f} minutes on {max_by_time["train_id"]}')
print(f'\tOn average trains take: {avg_time/60:.1f} minutes')
print(f'\tMedian time of trips: {median_time/60:.1f} minutes')
# maybe some sort of normal graph



	Shortest Trip (time wise): 17.0 minutes on 1751
	Longest Trip (time wise): 139.0 minutes on 1575
	On average trains take: 77.1 minutes
	Median time of trips: 71.0 minutes


In [12]:

if (railroad != "rtd"):
    # Get stops and their info
    df = pd.read_csv(f'../data/csv/{railroad}.csv') # assuming running form root
    df_ref = pd.read_csv(f'../gtfs_data/{railroad}/stops.txt')
    # df_ace = pd.read_csv(f'./gtfs_data/{ele}/shapes.txt')

    # Clean and calculate max trains per stop
    df.drop('Unnamed: 0', axis=1, inplace=True)
    row_stop_counts = df.drop(columns=["stop_name"]).notna().sum(axis=1)
    df_ref['counts'] = row_stop_counts
    df_ref = df_ref[['stop_name','counts']]
    df_ref = df_ref.sort_values(by='counts',ascending=False).iloc[0:3].reset_index(drop=True)
    print("\n\tTop three stations: ")
    for index,row in df_ref.iterrows():
        print(f'\t\t{index+1}. {row["stop_name"]} ({row["counts"]} trains)')
# graph already above



	Top three stations: 
		1. Grand Central (569 trains)
		2. Harlem-125 St (526 trains)
		3. Stamford (225 trains)


In [13]:
print("\n\tTime frequency (leaving times): ")
print(f'\t4 am to 10 am: {binned_times["4-10"]}')
print(f'\t10 am to 4 pm: {binned_times["10-16"]}')
print(f'\t4 pm to 10 pm: {binned_times["16-22"]}')
print(f'\t10pm to 4 am: {binned_times["22-4"]}')
print(f'\n')


	Time frequency (leaving times): 
	4 am to 10 am: 222
	10 am to 4 pm: 153
	4 pm to 10 pm: 229
	10pm to 4 am: 60




In [14]:
hourly_times
#  hourly_times[f'{timeA.hour}_{train["direction_id"]}']
hourly_times_df = pd.DataFrame(list(hourly_times.items()), columns=["hour", "value"])


# Split the lists in the 'value' column into separate columns
df_split = hourly_times_df["value"].apply(pd.Series)

# Rename the new columns
df_split.columns = ["value_0", "value_1"]

# Concatenate the new columns back with the 'hour' column
hourly_times_df = pd.concat([hourly_times_df["hour"], df_split], axis=1)


hourly_times_df["hour"] = hourly_times_df["hour"].astype(int)
hourly_times_df['total'] = hourly_times_df['value_0'] +  hourly_times_df['value_1']
hourly_times_df.sort_values(by='hour')
hourly_times_df = hourly_times_df.set_index("hour").reindex(range(24), fill_value=0).reset_index()
fig = px.bar(hourly_times_df, 
             x="hour", 
             y=["total"],
              title="Bar Graph of Hourly Values",
              text_auto=True,
              barmode="group")

# # Show the figure
fig.show()


In [15]:
hourly_times
#  hourly_times[f'{timeA.hour}_{train["direction_id"]}']
hourly_times_df = pd.DataFrame(list(hourly_times.items()), columns=["hour", "value"])


# Split the lists in the 'value' column into separate columns
df_split = hourly_times_df["value"].apply(pd.Series)

# Rename the new columns
df_split.columns = ["value_0", "value_1"]

# Concatenate the new columns back with the 'hour' column
hourly_times_df = pd.concat([hourly_times_df["hour"], df_split], axis=1)


hourly_times_df["hour"] = hourly_times_df["hour"].astype(int)
hourly_times_df.sort_values(by='hour')
hourly_times_df = hourly_times_df.set_index("hour").reindex(range(24), fill_value=0).reset_index()
fig = px.bar(hourly_times_df, 
             x="hour", 
             y=["value_0", "value_1"],
              title="Bar Graph of Hourly Values",
              text_auto=True,
              barmode="group")

# # Show the figure
fig.show()


In [16]:

#  hourly_times[f'{timeA.hour}_{train["direction_id"]}']
hourly_times_df = pd.DataFrame(list(hourly_times_arriving.items()), columns=["hour", "value"])


# Split the lists in the 'value' column into separate columns
df_split = hourly_times_df["value"].apply(pd.Series)

# Rename the new columns
df_split.columns = ["value_0", "value_1"]

# Concatenate the new columns back with the 'hour' column
hourly_times_df = pd.concat([hourly_times_df["hour"], df_split], axis=1)


hourly_times_df["hour"] = hourly_times_df["hour"].astype(int)
hourly_times_df.sort_values(by='hour')
hourly_times_df = hourly_times_df.set_index("hour").reindex(range(24), fill_value=0).reset_index()
fig = px.bar(hourly_times_df, 
             x="hour", 
             y=["value_0", "value_1"],
              title="Bar Graph of Hourly Values",
              text_auto=True,
              barmode="group")

# # Show the figure
fig.show()


In [17]:
df = pd.DataFrame(num_stops)
df = df[df['distance'] != "NA"]
df['travel_time'] = df['travel_time'] / 60
xaxis_max = df['distance'].max() + 5
yaxis_max = df['travel_time'].max() + 5
fig = px.scatter(df, x="distance", y="travel_time", color='number_of_stops', hover_name="train_label",
                #   trendline="rolling", trendline_options=dict(window=10),
                 )
fig.update_layout(
    xaxis_range = [0,xaxis_max],
    yaxis_range = [0,yaxis_max],
    width = 1400,
    height = 700
)
fig.show()

In [18]:
df = pd.DataFrame(num_stops)
df = df[df['distance'] != "NA"]
df['travel_time'] = df['travel_time'] / 60
xaxis_max = df['distance'].max() + 5
yaxis_max = df['travel_time'].max() + 5
fig = px.scatter(df, x="distance", y="travel_time", color='train_line', hover_name="train_label",
                 )
fig.update_layout(
    xaxis_range = [0,xaxis_max],
    yaxis_range = [0,yaxis_max],
    width = 1400,
    height = 700
)
fig.show()

In [19]:
df = pd.DataFrame(num_stops)
df["travel_time"] = df["travel_time"] / 60  # Convert to hours

global_min = df["number_of_stops"].min()
global_max = df["number_of_stops"].max()
xaxis_max = df['distance'].max() + 5
yaxis_max = df['travel_time'].max() + 5
# Get unique train lines
train_lines = df["train_line"].unique()

# Create traces for each train line
fig = go.Figure()

# Define the colorscale for the gradient
colorscale = px.colors.sequential.Plasma

for line in train_lines:
    filtered_df = df[df["train_line"] == line]
    fig.add_trace(
        go.Scatter(
            x=filtered_df["distance"],
            y=filtered_df["travel_time"],
            mode="markers",
            marker=dict(
                color=filtered_df["number_of_stops"], 
                colorscale=colorscale,
                cmin=global_min,  
                cmax=global_max,  
                colorbar=dict(
                    title="Number of Stops",
                ),
            ),
            name=line, 
            hovertext=filtered_df["train_label"],
            visible=True, 
        )
    )


# Add dropdown buttons
buttons = [
    dict(
        label=line,
        method="update",
        args=[
            {"visible": [line == train for train in train_lines]}, 
            {"title": f"Train Line: {line}"},
        ],
    )
    for line in train_lines
]
buttons.insert(
    0,
    dict(
        label="All",
        method="update",
        args=[
            {"visible": [True] * len(train_lines)},  
            {"title": "All Train Lines"},
        ],
    )
)

# Update layout
fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            showactive=True,
        )
    ],
    title="Train Line Filter with Color Gradient",
    xaxis_range = [0,xaxis_max],
    yaxis_range = [0,yaxis_max],
    xaxis_title="Distance (miles)",
    yaxis_title="Travel Time (minutes)",
    height = 700,
    showlegend=False
)

fig.show()


In [20]:
df = pd.DataFrame(num_stops)
df = df[df['distance'] != "NA"]
df['travel_time'] = df['travel_time'] / 60
xaxis_max = df['distance'].max() + 5
yaxis_max = df['travel_time'].max() + 5
fig = px.scatter_3d(df, x='distance', y='travel_time', z='number_of_stops',
                    color='number_of_stops', symbol='train_line')
fig.show()

In [21]:

if (railroad != "rtd"):
    # Get stops and their info
    df = pd.read_csv(f'../data/csv/{railroad}.csv') # assuming running form root
    df_ref = pd.read_csv(f'../gtfs_data/{railroad}/stops.txt')
    # df_ace = pd.read_csv(f'./gtfs_data/{ele}/shapes.txt')

    # Clean and calculate max trains per stop
    df.drop('Unnamed: 0', axis=1, inplace=True)
    row_stop_counts = df.drop(columns=["stop_name"]).notna().sum(axis=1)
    df_ref['counts'] = row_stop_counts
    df_ref = df_ref[['stop_name','counts']]


fig = px.bar(df_ref, x="stop_name", y=["counts"])
fig.show()

In [41]:
df = pd.read_csv(f'../data/csv/{railroad}.csv').drop('Unnamed: 0',axis=1)
df = df.set_index('stop_name')
df = df.filter(like="Harlem")
df = df.notna()
df = df.T
df_filtered = df.loc[:, ~df.all()] 
print(df_filtered)

# data_numeric = df.to_numpy().astype(int)

# import matplotlib.pyplot as plt
# import numpy as np

# # Plot using pcolormesh
# plt.figure(figsize=(15, 10))  # Set a custom figure size to control aspect ratio
# plt.pcolormesh(data_numeric, cmap="binary", edgecolors="none", linewidth=0)

# # Add titles and labels
# plt.title("True/False Grid with Custom Aspect Ratio")
# plt.axis("off")  # Turn off the axes for a cleaner grid
# plt.show()


# Create an annotated heatmap


fig = px.imshow(df_filtered.astype(int),  # Convert True/False to 1/0 for visualization
                color_continuous_scale=["white", "black"],  # White for False, Black for True
                title="True/False Grid Visualization")
fig.update_layout(
    coloraxis_showscale=False,  # Hide the color bar if not needed
    width=1000,                # Set the width of the plot
    height=4000                # Match height to width for a square grid
)
fig.show()

In [23]:
from plotly import data
df = data.gapminder()
df = df.loc[(df.continent == "Americas") & (df.year.isin([1952, 2002]))]

countries = (
    df.loc[(df.continent == "Americas") & (df.year.isin([2002]))]
    .sort_values(by=["lifeExp"], ascending=True)["country"]
    .unique()
)

data = {"line_x": [], "line_y": [], "1952": [], "2002": [], "colors": [], "years": [], "countries": []}

for country in countries:
    data["1952"].extend([df.loc[(df.year == 1952) & (df.country == country)]["lifeExp"].values[0]])
    data["2002"].extend([df.loc[(df.year == 2002) & (df.country == country)]["lifeExp"].values[0]])
    data["line_x"].extend(
        [
            df.loc[(df.year == 1952) & (df.country == country)]["lifeExp"].values[0],
            df.loc[(df.year == 2002) & (df.country == country)]["lifeExp"].values[0],
            None,
        ]
    )
    data["line_y"].extend([country, country, None]),

fig = go.Figure(
    data=[
        go.Scatter(
            x=data["line_x"],
            y=data["line_y"],
            mode="lines",
            showlegend=False,
            marker=dict(
                color="grey"
            )
        ),
        go.Scatter(
            x=data["1952"],
            y=countries,
            mode="markers",
            name="1952",
            marker=dict(
                color="green",
                size=10
            )

        ),
        go.Scatter(
            x=data["2002"],
            y=countries,
            mode="markers",
            name="2002",
            marker=dict(
                color="blue",
                size=10
            )
        ),
    ]
)

fig.update_layout(
    title=dict(text="Life Expectancy in Europe: 1952 and 2002"),
    height=1000,
    legend_itemclick=False
)

fig.show()